In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import networkx as nx
import numpy as np
from scipy.sparse.linalg import eigs
import scipy.sparse as sp

from randcolorgraphs.utils.calculate_katz import calculate_katz

def get_edge_vector_from_adj_matrix(adj_matrix):
    sparse_matrix = sp.csr_matrix(adj_matrix)
    row_indices, col_indices = sparse_matrix.nonzero()
    edge_vector = np.vstack((row_indices, col_indices)).T
    return edge_vector

def generate_adjacency_matrix_and_katz_vector(n, p, seed=42, alpha=0.1, beta=1.0):
    G = nx.erdos_renyi_graph(n, p, directed=True, seed=seed)
    A_G = nx.adjacency_matrix(G).astype(np.float64)


    eigenvalues, _ = eigs(A_G, k=1, which='LM')  # 'LM': Largest Magnitude, tol is tolerance
    spectral_radius_G = np.abs(eigenvalues).max()
    print(f"Spectral Radius: {spectral_radius_G}, alpha_max = {1/spectral_radius_G}")
    print("calculating katz")
    katz_centrality = calculate_katz(A_G, alpha = alpha, beta=beta)
    print("done calculating katz")
    # Reorder nodes based on Katz centrality
    sorted_indices = np.argsort(katz_centrality)
    sorted_nodes = np.array(G.nodes())[sorted_indices]

    # Generate the adjacency matrix
    adj_matrix = nx.adjacency_matrix(G, nodelist=sorted_nodes)

    sorted_katz_vector = katz_centrality[sorted_indices]

    assert np.allclose(sorted_katz_vector, calculate_katz(adj_matrix.astype(np.float64), alpha = alpha, beta=beta))
    return adj_matrix, sorted_katz_vector


In [ ]:
n = 500
p = 6/n
adj_matrix, katz_vector = generate_adjacency_matrix_and_katz_vector(n=n, p=p, alpha=0.1)
edges = get_edge_vector_from_adj_matrix(adj_matrix)

In [10]:
from randcolorgraphs.algorithms.linear_scalarization.unoptimized_greedy_search import unoptimized_greedy_search_linear_scalarization
w = np.exp(-9)

# Can go to n ~ 500; smaller w -> much slower
clusters, _ = unoptimized_greedy_search_linear_scalarization(katz_vector, adj_matrix.todense(), np.array([0]*len(katz_vector), dtype=np.int64), w=w, pam_cluster_dist=2)

init obj: 188.74177060819267
Iteration 0 Objective: 65.20547986597174 move_type split
Iteration 1 Objective: 41.2349371806622 move_type split
Iteration 2 Objective: 20.920409132332757 move_type split
Iteration 3 Objective: 15.969978419832403 move_type split
Iteration 4 Objective: 12.590356281151568 move_type split
Iteration 5 Objective: 9.429311742360872 move_type split
Iteration 6 Objective: 6.379550262260105 move_type split
Iteration 7 Objective: 5.3117655854531804 move_type split
Iteration 8 Objective: 4.520067846691405 move_type split
Iteration 9 Objective: 3.766197541873956 move_type split
Iteration 10 Objective: 3.211760190959291 move_type split
Iteration 11 Objective: 2.7295689517501724 move_type split
Iteration 12 Objective: 2.3059943809687002 move_type split
Iteration 13 Objective: 1.9116668883222103 move_type split
Iteration 14 Objective: 1.6925232099356904 move_type split
Iteration 15 Objective: 1.5218253095033798 move_type split
Iteration 16 Objective: 1.364911824865482 mov

In [11]:
from randcolorgraphs.algorithms.linear_scalarization.optimal_contiguous.optimal_contiguous_linear_scalarization_algo import optimal_contiguous_linear_scalarization_algo

# Can go up to n ~ 2_000; indepedented of w
clusters = optimal_contiguous_linear_scalarization_algo(katz_vector, adj_matrix.todense(), w)

In [12]:
from greedy_algo_fast import greedy_search
# Can go up to n ~ 1_000_000;
clusters = greedy_search(katz_vector, edges, np.array([0]*len(katz_vector)), w=w, max_interaction_dist=1)

Iteration 0 Objective: 65.20547986597009 move_type split
Iteration 1 Objective: 41.234937180662605 move_type split
Iteration 2 Objective: 20.920409132337024 move_type split
Iteration 3 Objective: 15.969978419835986 move_type split
Iteration 4 Objective: 12.590356281153557 move_type split
Iteration 5 Objective: 9.42931174236059 move_type split
Iteration 6 Objective: 6.379550262260106 move_type split
Iteration 7 Objective: 5.3117655854533234 move_type split
Iteration 8 Objective: 4.52006784669098 move_type split
Iteration 9 Objective: 3.7661975418729616 move_type split
Iteration 10 Objective: 3.2117601909580413 move_type split
Iteration 11 Objective: 2.7295689517490365 move_type split
Iteration 12 Objective: 2.30599438096819 move_type split
Iteration 13 Objective: 1.911666888321785 move_type split
Iteration 14 Objective: 1.692523209934981 move_type split
Iteration 15 Objective: 1.521825309502443 move_type split
Iteration 16 Objective: 1.364911824864005 move_type split
Iteration 17 Object